In [1]:
import os
import urllib
import zipfile
import time

import numpy as np
import pandas as pd

In [4]:
data_dir = "data"
!mkdir $data_dir

!cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!cd $data_dir && unzip ml-latest-small.zip
dataset_dir = data_dir + "/ml-latest-small/"
!ls $dataset_dir

--2023-12-01 17:33:08--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org... 128.101.65.152
Connecting to files.grouplens.org|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: 'ml-latest-small.zip'

ml-latest-small.zip 100%[===================>] 955.28K  1.12MB/s    in 0.8s    

2023-12-01 17:33:09 (1.12 MB/s) - 'ml-latest-small.zip' saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
README.txt  links.csv   movies.csv  ratings.csv tags.csv


In [2]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [3]:
movie_df = pd.read_csv('data/ml-latest-small/movies.csv')
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
merged_df = pd.merge(ratings, movie_df, on='movieId', how='inner')
merged_df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


# Movie Info

Find out top 10 movies that have been watched and rated the most

In [5]:
movie_popularity = merged_df.groupby('title').size().sort_values(ascending=False)
movie_popularity.head(10)

title
Forrest Gump (1994)                          329
Shawshank Redemption, The (1994)             317
Pulp Fiction (1994)                          307
Silence of the Lambs, The (1991)             279
Matrix, The (1999)                           278
Star Wars: Episode IV - A New Hope (1977)    251
Jurassic Park (1993)                         238
Braveheart (1995)                            237
Terminator 2: Judgment Day (1991)            224
Schindler's List (1993)                      220
dtype: int64

Top 10 movies that has the highest rating with over 50 counts of ratings

In [6]:
min_ratings = 50
avg_ratings = merged_df.groupby('title')['rating'].mean()
rating_counts = merged_df.groupby('title')['rating'].count()
popular_movies = avg_ratings[rating_counts > min_ratings].sort_values(ascending=False)
popular_movies.head(10)

title
Shawshank Redemption, The (1994)                                               4.429022
Godfather, The (1972)                                                          4.289062
Fight Club (1999)                                                              4.272936
Cool Hand Luke (1967)                                                          4.271930
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)    4.268041
Rear Window (1954)                                                             4.261905
Godfather: Part II, The (1974)                                                 4.259690
Departed, The (2006)                                                           4.252336
Goodfellas (1990)                                                              4.250000
Casablanca (1942)                                                              4.240000
Name: rating, dtype: float64

# User Info

In [7]:
user_movie_matrix = merged_df.pivot_table(index='userId', columns='title', values='rating')
user_movie_matrix = user_movie_matrix.fillna(0)
user_movie_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,3.5,0.0,0.0,0.0


Cosine Similarity

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(user_movie_matrix)
similarity_df = pd.DataFrame(similarity_matrix, index=user_movie_matrix.index, columns=user_movie_matrix.index)

Find top 20 similar user

In [9]:
user_id = 134  # Replace with any userID
similar_users = similarity_df[user_id].sort_values(ascending=False)
print(similar_users.head(20))

userId
134    1.000000
46     0.626597
512    0.553572
126    0.543569
353    0.539496
468    0.535917
56     0.535311
584    0.531455
574    0.524055
102    0.522913
130    0.503191
592    0.497560
242    0.492093
145    0.478892
340    0.478459
176    0.477550
379    0.466675
447    0.453111
94     0.449846
404    0.446166
Name: 134, dtype: float64


Define a function that gather the movie and ratings history from a particular user for training (return 80% of data)

In [20]:
def get_user_watch_history(user_id, data, fraction=0.8):
    user_watch_history = data[data['userId'] == user_id]
    user_history_train = user_watch_history.sample(frac=fraction, random_state=42)

    return user_history_train

user_history_train = get_user_watch_history(user_id=1, data=merged_df)
user_history_train = user_history_train[['rating','title','genres']]
print(user_history_train.head(10))

       rating                                            title  \
15656     5.0                                 Gladiator (2000)   
6579      4.0                                Abyss, The (1989)   
1135      5.0                            Canadian Bacon (1995)   
12997     2.0                                Mummy, The (1999)   
1568      5.0        Star Wars: Episode IV - A New Hope (1977)   
15028     5.0  Messenger: The Story of Joan of Arc, The (1999)   
2998      5.0                             Fugitive, The (1993)   
14755     4.0                      Being John Malkovich (1999)   
12182     4.0                       Romancing the Stone (1984)   
11074     5.0                           Negotiator, The (1998)   

                                                genres  
15656                           Action|Adventure|Drama  
6579                  Action|Adventure|Sci-Fi|Thriller  
1135                                        Comedy|War  
12997  Action|Adventure|Comedy|Fantasy|Horror